# Summary
<br>
Code based on https://arxiv.org/abs/1902.09833 
<br> Simulating a two level system, emmiting a photon into a cavity using, and the superoperator of this system in a reduced basis 9x9
<br>

---

In [13]:
from sympy import symbols
import qutip
import numpy as np
import matplotlib.pyplot as plt

In [14]:
import sympy as sp
from math import factorial as fact

In [53]:
from sympy import latex

In [17]:
g = symbols('g ', real=True)
y = symbols('y', positive=True)                      #reprsents gamma
t = symbols('t', positive=True)                 


In [42]:
d = symbols('d')
dt = symbols('dt')
L = symbols('L')


L

In [31]:
m1 , m2, m3 = symbols('m_1'), symbols('m_2'), symbols('m_3')
m4 , m5, m6 = symbols('m_4'), symbols('m_5'), symbols('m_6')
m7 , m8, m9 = symbols('m_7'), symbols('m_8'), symbols('m_9')
rho_vec = sp.Matrix(9,1,[m1, m2, m3, m4, m5, m6, m7, m8, m9])
rho_mat = sp.Matrix(3,3,[m1, m2, m3, m4, m5, m6, m7, m8, m9])

In [19]:
basis = list()
for i in range(9):
    test_b = [0,]*9
    test_b[i] = 1
    basis.append(sp.Matrix(3,3,test_b))

In [20]:
basis[0]          #example

Matrix([
[1, 0, 0],
[0, 0, 0],
[0, 0, 0]])

In [21]:
c = sp.Matrix(3,3,[0,0,1,0,0,0,0,0,0])
c_d = c.conjugate().T

a = sp.Matrix(3,3,[0,1,0,0,0,0,0,0,0])
a_d = a.conjugate().T

L0 = sp.sqrt(y)*c + g*a 
L0_d = L0.conjugate().T

H = 1j*0.5*g* sp.sqrt(y) *(c_d@a - a_d@c)

In [22]:
#Superoperator application function

def L(m):
    #d_rho = -1j*( (H@m) - (m@H) )  +  (L0@m@L0_d) - 0.5*( (L0_d@L0@m) + (m@L0_d@L0) ) 
    d_rho = 1j*m@H - 1j*H@m   +  L0@m@L0_d - 0.5*L0_d@L0@m - 0.5*m@L0_d@L0 
    return d_rho

In [23]:
L(basis[0])              #example

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

In [24]:
#calculating the superoperator

super_op = 0*sp.eye( 9 )
for i in range(9):
    for j in range(9):
        super_op[i,j] = sp.trace( basis[i].T @ L(basis[j]) )

super_op = sp.simplify(super_op)

In [26]:
super_op

Matrix([
[0,         0,              0,         0,      g**2,         g*sqrt(y),              0,         g*sqrt(y),              y],
[0, -0.5*g**2, -1.0*g*sqrt(y),         0,         0,                 0,              0,                 0,              0],
[0,         0,         -0.5*y,         0,         0,                 0,              0,                 0,              0],
[0,         0,              0, -0.5*g**2,         0,                 0, -1.0*g*sqrt(y),                 0,              0],
[0,         0,              0,         0, -1.0*g**2,    -1.0*g*sqrt(y),              0,    -1.0*g*sqrt(y),              0],
[0,         0,              0,         0,         0, -0.5*g**2 - 0.5*y,              0,                 0, -1.0*g*sqrt(y)],
[0,         0,              0,         0,         0,                 0,         -0.5*y,                 0,              0],
[0,         0,              0,         0,         0,                 0,              0, -0.5*g**2 - 0.5*y, -1.0*g*sqrt(y)],

---

basis : 
<br> |0>|0> = $\begin{pmatrix}
1\\0\\0\\
\end{pmatrix}$
<br> |0>|1> = $\begin{pmatrix}
0\\1\\0\\
\end{pmatrix}$
<br> |1>|0> = $\begin{pmatrix}
0\\0\\1\\
\end{pmatrix}$

$$\frac{d}{dt}\rho = \mathcal{L}(\rho)$$

$$\rho = \begin{pmatrix}
m_{1}&m_{2}&m_3\\
m_4&m_5&m_6\\
m_7&m_8&m_9\\
\end{pmatrix}$$

$$\frac{d}{dt}\rho = \frac{d}{dt}\begin{pmatrix}
m_{1}\\m_{2}\\m_3\\
m_4\\m_5\\m_6\\
m_7\\m_8\\m_9\\
\end{pmatrix} = \mathcal{L}(\begin{pmatrix}
m_{1}&m_{2}&m_3\\
m_4&m_5&m_6\\
m_7&m_8&m_9\\
\end{pmatrix})$$

---

In [57]:
(d/dt)*rho_vec - sp.simplify(super_op*rho_vec)                           #is equal to zero

Matrix([
[d*m_1/dt - g**2*m_5 - g*m_6*sqrt(y) - g*m_8*sqrt(y) - m_9*y],
[                 d*m_2/dt + g*(0.5*g*m_2 + 1.0*m_3*sqrt(y))],
[                                       d*m_3/dt + 0.5*m_3*y],
[                 d*m_4/dt + g*(0.5*g*m_4 + 1.0*m_7*sqrt(y))],
[       d*m_5/dt + 1.0*g*(g*m_5 + m_6*sqrt(y) + m_8*sqrt(y))],
[          d*m_6/dt + 1.0*g*m_9*sqrt(y) + 0.5*m_6*(g**2 + y)],
[                                       d*m_7/dt + 0.5*m_7*y],
[          d*m_8/dt + 1.0*g*m_9*sqrt(y) + 0.5*m_8*(g**2 + y)],
[                                       d*m_9/dt + 1.0*m_9*y]])

with intial conditions: $\rho (0) = \begin{pmatrix}
0&0&0\\
0&0&0\\
0&0&1\\
\end{pmatrix}$
<br> $g_v(t) = \frac{-1}{\sqrt{e^{\gamma t}-1}}$

$$m_9 = e^{-t\gamma}$$
$$m_7, m_3 = 0$$
$$m_2, m_4 = 0$$

$$m_8 = m_6$$
<br> $$\frac{d}{dt}m_8 = -m_8(\frac{g^2_v(t)+\gamma}{2}) - g_v(t)\sqrt{\gamma}e^{-t\gamma}$$
<br> Solving this :
<br> $$m_8 = \frac{2\sqrt{\gamma}}{\gamma + 1} \sqrt{e^{-t\gamma}(1-e^{-t\gamma})}$$

 $$m_5 = \frac{-4\gamma}{(\gamma + 1)^2} (1-e^{-t\gamma})$$

 $$m_1 = (1 - \frac{4}{\gamma+1} - \frac{4}{(\gamma+1)^2}) (1-e^{-t\gamma})$$

however : 
$$Tr(\rho) = m_1 + m_5 + m_8 = 1 - \frac{8}{\gamma +1}(1-e^{-t\gamma})$$